In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("hive-test") \
    .master("spark://spark-master:7077")\
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [9]:
#Use WES Database
spark.sql("USE hw3")
current_db = spark.sql("SELECT current_database()").collect()[0][0]
print(current_db)

hw3


In [ ]:
# df = spark.read.csv("clean/world.csv", header=True, inferSchema=True)

In [4]:
# df.write.saveAsTable("wes.world")
from utils import run_spark_sql

In [31]:

sql = """
WITH gold_medals_by_discipline AS (
    SELECT
        t.discipline AS sport,
        t.event,
        COUNT(*) AS gold_medal_count
    FROM hw3.teams t
    JOIN hw3.medals m ON t.NOC = m.team
    WHERE m.gold > 10
    GROUP BY t.discipline, t.event
)

SELECT
    sport,
    event,
    gold_medal_count
FROM gold_medals_by_discipline
ORDER BY gold_medal_count DESC
LIMIT 5
"""


sql = """
SELECT t.discipline, COUNT(*) AS discipline_count
FROM teams t
JOIN medals m ON t.noc = m.team
WHERE m.gold > 10
GROUP BY t.discipline
ORDER BY discipline_count DESC
LIMIT 5;
"""
data = spark.sql(sql)
data.show()

# sql = "SELECT * FROM teams"
# data = spark.sql(sql)
# data.head(5)

sql = "SELECT * FROM medals"
data = spark.sql(sql)
data.head(5)


+-------------+----------------+
|   discipline|discipline_count|
+-------------+----------------+
|     Swimming|              40|
|      Fencing|              20|
|Cycling Track|              18|
|      Archery|              16|
|    Athletics|              16|
+-------------+----------------+



[Row(rank=1, team='United States of America', gold=39, silver=41, bronze=33, total=113, total_rank=1),
 Row(rank=2, team="People's Republic of China", gold=38, silver=32, bronze=18, total=88, total_rank=2),
 Row(rank=3, team='Japan', gold=27, silver=14, bronze=17, total=58, total_rank=5),
 Row(rank=4, team='Great Britain', gold=22, silver=21, bronze=22, total=65, total_rank=4),
 Row(rank=5, team='ROC', gold=20, silver=28, bronze=23, total=71, total_rank=3)]

In [36]:
sql = """
-- Question 4: Return the list of countries and their rank by total who participate in the Basketball discipline
WITH basketball_participation AS (
    SELECT
        DISTINCT
        t.NOC AS team,
        m.total AS total_medals
    FROM hw3.teams t
    LEFT JOIN hw3.medals m ON t.NOC = m.team
    WHERE t.discipline = 'Basketball'
)

SELECT
    team,
    total_medals,
    RANK() OVER (ORDER BY total_medals DESC) AS rank_by_total
FROM basketball_participation
ORDER BY rank_by_total;
"""

# sql = "SELECT * FROM medals"
data = spark.sql(sql)
data.show(30)



+--------------------+------------+-------------+
|                team|total_medals|rank_by_total|
+--------------------+------------+-------------+
|United States of ...|         113|            1|
|People's Republic...|          88|            2|
|               Japan|          58|            3|
|           Australia|          46|            4|
|               Italy|          40|            5|
|             Germany|          37|            6|
|              France|          33|            7|
|              Canada|          24|            8|
|   Republic of Korea|          20|            9|
|               Spain|          17|           10|
|      Czech Republic|          11|           11|
|              Serbia|           9|           12|
|             Belgium|           7|           13|
|Islamic Republic ...|           7|           13|
|            Slovenia|           5|           15|
|           Argentina|           3|           16|
|             Nigeria|           2|           17|
